<a href="https://colab.research.google.com/github/parikher/Diacare/blob/master/NLP-Yake-KPE-200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import os
import pandas as pd

!pip install rake_nltk
from rake_nltk import Metric, Rake
import nltk
import string
string.punctuation
stopword = nltk.corpus.stopwords.words('english')
!pip install yake
import yake
import re
 

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 81kB 4.1MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 245kB 46.7MB/s 
  Created wheel for yake: filename=yake-0.3.7-py2.py3-none-any.whl size=44591 sha256=a2f2eb17c455dfc1df82ee73072f706c3d4eb0317de6138bce3be8e792cc8948
  Stored in directory: /root/.cache/pip/wheels/90/7d/56/63a3a3b064e9214e29616880dc4170a300ca68643728ce24ba
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=066dab05c926fb004f67e7fbc019cd4cc56a59d03fea4c9c4a22db30d8c2b499
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25021 sha256=290fd91fe953d56f5383acd66744a340c5e490c073769337c0d0dc58c4b4c6cc
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
Success

In [4]:
#Load Part of speech tags .. standard from stanford uni NLP lab .. using this to remove tags from tags
pos_tags = ['/CC ','/CD ','/DT ','/EX ','/FW ','/IN ','/JJ ','/JJR ','/JJS ','/LS ','/MD ','/NN ','/NNP ','/NNPS ','/NNS ','/PDT ','/POS ','/PRP ','/PRP$ ','/prp$ ','/punct ','/RB ','/RBR ','/rbs ','/RP ','/SYM ','/TO ','/UH ','/VB ','/VBD ','/VBG ','/VBN ','/VBP ','/VBZ ','/WDT ','/WP ','/WP$ ','/WRB ']

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [7]:
# Load Data to dataframe
# Read all files in the folder and save filename and data into dataframe
DataFolder = 'gdrive/My Drive/ColabNotebooks/dataset/ACM/'
df = pd.DataFrame(columns=['Doc-ID','Doc-Text'])
rows_list = []
for FileName in os.listdir(DataFolder):
    f = open(DataFolder + FileName, "r")
    Text_in_doc = f.read()
    Text_in_doc =  re.sub('\n',' ',Text_in_doc)
    dict1 = {}
    dict1[FileName]:Text_in_doc
    df = df.append({'Doc-ID': FileName,'Doc-Text' : Text_in_doc}, ignore_index=True)
df.shape

(2304, 2)

In [8]:
# PreProcessing data - remove tags
#Remove pos-tags from text and matching /prp$ /wp$ and replace brackets
def clear_text(text):
        for tag in pos_tags:
            text = re.sub(str(tag).lower(),' ',text)
            text = re.sub('/prp. ',' ',text)
            text = re.sub('/wp. ',' ',text)
            text = re.sub('-LRB-','(',text)
            text = re.sub('-RRB-',')',text)
            text = re.sub('-LSB-','[',text)
            text = re.sub('-RSB-',']',text)
        return text
df['AftClearText'] = df['Doc-Text'].apply(lambda x : clear_text(x))

In [9]:
#extract KPs with Yake
from tqdm import tqdm
tqdm.pandas()

kw_extractor = yake.KeywordExtractor(n=3,dedupLim=0.9,top=200,features=None )
def KPs_from_yake(text):
    KPsFrmYake = kw_extractor.extract_keywords(text)
    result = []
    for each in KPsFrmYake:
        result.append(each[1])
    return result


In [10]:
df['KPs_frm_YAKE'] = df['AftClearText'].progress_apply(lambda x : KPs_from_yake(x))  

100%|██████████| 2304/2304 [22:28<00:00,  1.71it/s]


In [12]:
# Visualise data in dataframe
df.head(5)

,Doc-ID,Doc-Text,AftClearText,KPs_frm_YAKE
0,626779.txt.final.pre,--/punct T/nn Fault/nnp Injection/nn and/cc De...,-- T Fault Injection and Dependability Evaluat...,"[fault injection experiments, fault injection,..."
1,176519.txt.final.pre,--/punct T/nn Precise/jj and/cc efficient/jj g...,-- T Precise and efficient groundness analysis...,"[boolean functions, positive boolean functions..."
2,323812.txt.final.pre,--/punct T/nn Active/jj Management/nn of/in Da...,-- T Active Management of Data Caches by Explo...,"[nts pcs nts, pcs nts pcs, mat nts pcs, nts pc..."
3,277910.txt.final.pre,--/punct T/nn Modeling/vbg set/vbn associative...,-- T Modeling set associative caches behavior ...,"[sparse matrix, sparse matrix-vector product, ..."
4,344994.txt.final.pre,--/punct T/nn Inexact/nnp Preconditioned/nnp C...,-- T Inexact Preconditioned Conjugate Gradient...,"[preconditioned conjugate gradient, inexact pr..."


In [13]:
# Take Doc-ID and KPs_frm_RAKE two columns two new data frame for further processing
df1 = df 
df1 = df1.drop(['Doc-Text', 'AftClearText'], axis=1)
df1.head(2)

,Doc-ID,KPs_frm_YAKE
0,626779.txt.final.pre,"[fault injection experiments, fault injection,..."
1,176519.txt.final.pre,"[boolean functions, positive boolean functions..."


In [14]:
# Load Ground Truth KPs from JSON to dataframe dftruth
import json
TruthFolder = 'gdrive/My Drive/ColabNotebooks/dataset/truth/'
dftruth = pd.DataFrame(columns=['Doc-ID-only','KPs_Truth'])


with open(TruthFolder + 'test.author.json') as json_file:
    data = json.load(json_file)
    for p in data:
        dftruth = dftruth.append({'Doc-ID-only': p,'KPs_Truth' : data[p]}, ignore_index=True)

In [15]:
dftruth = dftruth.sort_values(by=['Doc-ID-only'],ascending=False)
df1= df1.sort_values(by=['Doc-ID'],ascending=False)
dftruth = dftruth.reset_index(drop=True)
df1 = df1.reset_index(drop=True)
 

In [16]:
# Take dfa dfb for further processing 
dfa = df1
dfb = dftruth

# remove .txt.final.pre string from ID
dfa['Doc-ID'] = dfa['Doc-ID'].apply(lambda x : re.sub('.txt.final.pre','',x))  

# # sort both df by ID 
# dfa = dfa.sort_values(by=['Doc-ID'],ascending=True)
# dfb = dfb.sort_values(by=['Doc-ID-only'],ascending=True)

# Join two dfs by column - KPs from rake, KPs from ground truth and Unique ID
dfc = pd.concat([dfa,dfb], axis=1, ignore_index=False)

# Rearrange the columns as below
dfd = dfc[['Doc-ID','Doc-ID-only','KPs_Truth','KPs_frm_YAKE']]

dfd.head(2)

,Doc-ID,Doc-ID-only,KPs_Truth,KPs_frm_YAKE
0,637759,637759,"[[devs models], [applications of devs methodol...","[window invalid mask, trap base register, circ..."
1,637568,637568,"[[breakpoint analysis], [genome rearrangement]...","[pairwise inversion distance, maximum pairwise..."


In [17]:
dfb.head(2)

,Doc-ID-only,KPs_Truth
0,637759,"[[devs models], [applications of devs methodol..."
1,637568,"[[breakpoint analysis], [genome rearrangement]..."


In [18]:
# Calculate matching KPs percentage Matching of Ground Truth with Rake extracted KPs

def calculate_percent_found(row):
    rakeKPs = dfd.iloc[row,3]
    truthKPs =  dfd.iloc[row,2]
    total = len(truthKPs)
    found = 0
    for KP in truthKPs:
        if KP[0] in rakeKPs:
            found = found+1
    return (found/total)*100


# Add Index Column to dfd
dfd['index_col'] = df.index

# Apply calculate_percent_found function to have percent_found in separate column
dfd['percent_found'] = dfd['index_col'].apply(lambda x : calculate_percent_found(x))  
dfd.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Doc-ID,Doc-ID-only,KPs_Truth,KPs_frm_YAKE,index_col,percent_found
0,637759,637759,"[[devs models], [applications of devs methodol...","[window invalid mask, trap base register, circ...",0,25.000000
1,637568,637568,"[[breakpoint analysis], [genome rearrangement]...","[pairwise inversion distance, maximum pairwise...",1,33.333333


In [19]:
# Average KPs matching percentage for all documents 
Total = dfd['percent_found'].sum()
Total/2304

50.25634850373123

In [20]:
dfd.head(20)

,Doc-ID,Doc-ID-only,KPs_Truth,KPs_frm_YAKE,index_col,percent_found
0,637759,637759,"[[devs models], [applications of devs methodol...","[window invalid mask, trap base register, circ...",0,25.000000
1,637568,637568,"[[breakpoint analysis], [genome rearrangement]...","[pairwise inversion distance, maximum pairwise...",1,33.333333
2,637371,637371,"[[global optimization], [interval arithmetic],...","[interval analysis global, global optimization...",2,66.666667
3,637321,637321,"[[morphic predicates], [second-order]]","[residually ultimately periodic, ultimately pe...",3,50.000000
4,637247,637247,"[[ttl], [active probing], [bottleneck bandwidt...","[hop, ttl probe packet, invisible hops, clink ...",4,33.333333
5,637236,637236,"[[routing], [ospf], [lsa traffic], [enterprise...","[lsas day number, lsas change lsas, ospf lsa t...",5,75.000000
6,637151,637151,"[[fault-tolerance], [link failures], [cellular...","[defective cells, cell, cells, leftmost cell, ...",6,60.000000
7,636898,636898,"[[performance analysis], [tcp], [differentiate...","[average queue size, packet drop probability, ...",7,33.333333
8,636696,636696,"[[elliptic curves], [diffie-hellman], [authent...","[key agreement protocol, protocol, key, key ag...",8,50.000000
9,636581,636581,"[[path-following], [polytopes], [sperner's lem...","[full cells, full cell, full, full cell graph,...",9,50.000000


In [21]:
dfd.to_csv(path_or_buf = 'gdrive/My Drive/ColabNotebooks/dataset/truth/Yake200KPs.csv',index=False)